In [2]:
!pip install pyarrow
!pip install keras
!pip install tensorflow
!pip install ipynb
!pip install nbimporter

     |████████████████████████████████| 14.8 MB 24.9 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awswrangler 2.14.0 requires numpy<1.19.0,>=1.18.0; python_full_version ~= "3.6.2", but you have numpy 1.19.5 which is incompatible.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV
from itertools import combinations
import scipy.stats as stats
from sklearn.decomposition import PCA
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
from itertools import combinations
from sklearn.metrics import ConfusionMatrixDisplay
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

## define the function to do all the preprocessing in one line with iterable parameters

In [2]:
import nbimporter
import proccess_class as pc

In [49]:
def finale_preprocessing_data(data,
                             pct_nan,
                             correlation_treshold,
                             del_high_corr,
                             concentrated_treatment,
                             concent_treshold,
                             treat_outliers,
                             variables='todas',## we can pass todas so it takes all the data frame or the list of variables we want to use
                             variables_exclude=[],## we can exclude some variables from the dataset
                             perc_step=2.5, ## how many percent will be reach bin when dividing the dataset. 
                             correction_type='replace',## it can be replace or exlude depending on the method you want to use
                             exclude_negatives=False, ## if you want to replace the negatives with missing values
                             methodology='percentile', ## methodology could be "percentile" or "std"
                             std_integer=2,
                             escalador=StandardScaler(),
                             PCA_treatment=False,
                             number_components=10,
                             drop_vars=["visa_second_category","maplocation_name_frecuency",
                                        "ecosystem_name_frecuency","dep_second_category","departamento"],
                             cat_corr_tresh=0.8):
    """
    function to get the preprocessed data with a set of parameters.
    **params:
    data: the data set you want to be cleaned.
    pct_nan: percentaje of NaNs for a variable to be dropped.
    correlation_treshold: level of correlation to drop correlated numerical variables
    del_high_corr [bool][True,False]: define whether or not to drop variables with high correlations according to "correlation_treshold"
    concentrated_treatment [bool][True,False]: whether or not pass variables that are concentrated as one hot encoding
    concent_treshold:pct of concentration for a numerical variable to be passed as categorical.
    treat_outliers: decides whether or not treat the outlyers with diverse methodologies
    variables: we can pass todas so it takes all the data frame or the list of variables we want to use
    variables_exclude: we can exclude some variables from the dataset
    perc_step: how many percent will be each bin when dividing the dataset. 
    correction_type['replace','exclude']: it can be replace or exlude depending on the method you want to use
    exclude_negatives: if you want to replace the negatives with missing values
    methodology['percentile','std']: methodology could be "percentile" or "std"
    std_integer: how much standar deviation away from the mean should be the data to be taken as outlier.
    PCA_treatment[bool][False,True]: wheter or not apply PCA in numerical variables.
    escalador= the scaler we want to use for numerical variables, this should be de sklearn object
    number_components:numero de componentes a tener en cuenta en el pca
    drop_vars: categorical variables to drop from the database
    cat_corr_tresh: treshold to drop categorical variables with correlation
    """
    
    df_usuarios=data.copy(deep=True)
    #######################################

    ## use the function to drop columns with missing values.
    data_fil=pc.preproccessing_func.limpiar_data(df_usuarios,pct_nan)

    ## drop columns that dont improve the predictions
    data_fil.drop(columns=['user_id', 'document_number', 'user_birthdate', 'user_createddate', 'score_coink_quantile', 'score_coink_min_max' ],inplace=True)

    ## columns we don't need beacuse they are contained in other columns.
    columns=['temp_visa_spending_establishment_name','visa_currency_second_category']
    data_fil.drop(columns=columns,inplace=True)
    
    #######################

    ## inputing the variables we want to input
    data_fil['url_type'].fillna('otros',inplace=True)
    
    #########################################################################################
    col_numericas = ['account_type_id', 'current_money', 'number_of_deposits', 'number_of_withdrawals',
                   'tarjeta_visa_general_transacciones',
                   'tarjeta_visa_general', 'comfama',
                   'pocket_goals_count', 'score_coink', 'user_phonenumber','user_email',
                   'numero_cuentas', 'edad', 'antiguedad',
                   'rolling_180_cash_in_mean', 'rolling_180_cash_in_median',
                   'rolling_180_cash_in_max', 'rolling_180_deposit_num',
                   'rolling_180_cash_out_median', 'rolling_180_ret_num',
                   'rolling_180_total_cash_in', 'rolling_180_total_cash_out',
                   'num_dep_oink', 'cash_in_dep_oink', 'num_pse', 'cash_in_pse', 'num_p2p',
                   'cash_in_p2p', 'cash_out_p2p', 'num_otras_entidades_ACH',
                   'cash_out_otras_entidades_ACH', 'num_oper_visa', 'cash_out_oper_visa',
                   'num_remesas', 'cash_in_remesas', 'num_reval_ptm', 'cash_in_reval_ptm',
                   'cash_out_reval_ptm', 'visa_cards_number',
                   'distancia_black_flags', 'num_p2p_users', 'num_loc_points',
                   'cluster_number', 'numero_de_referidos','tran_seq_750000_7_0.1','tran_seq_300000_3_0.2',
                   'p2p_cash_in','tran_seq_500000_7_0.1','tran_seq_500000_5_0.2','tran_seq_750000_3_0.05',
                   'tran_seq_1000000_3_0.05','users_ip','oinks_pct_cash_in','tran_seq_750000_5_0.05',
                   'tran_seq_300000_7_0.1','tran_seq_1000000_7_0.05','tran_seq_1000000_3_0.1',
                   'tran_seq_750000_5_0.2','tran_seq_750000_7_0.2','tran_seq_500000_3_0.05',
                   'visa_cashout','tran_seq_500000_7_0.2','tran_seq_500000_5_0.05','pct_mean_withd_caj',
                   'time_diff_tran_norm_den','tran_seq_1000000_5_0.2','tran_seq_300000_5_0.1',
                   'tran_seq_300000_5_0.05','tran_seq_500000_7_0.05','tran_seq_750000_7_0.05',
                   'tran_seq_750000_3_0.2','tran_seq_1000000_3_0.2','tran_seq_750000_5_0.1',
                   'numero_sospechas_1','tran_seq_300000_3_0.05','no_anomalies_balance','tran_seq_1000000_5_0.1',
                   'tran_seq_750000_3_0.1','tran_seq_1000000_7_0.1','cajero_cashout','pse_cashin',
                   'tran_seq_300000_7_0.2','tran_seq_500000_5_0.1','tran_seq_1000000_5_0.05','tran_seq_500000_3_0.1',
                   'pct_balance_min','tran_seq_1000000_7_0.2','numero_sospechas','numero_rechazadas',
                   'pct_mean_withd','corresp_cashout','tran_seq_300000_5_0.2','tran_seq_300000_7_0.05',
                   'tran_seq_300000_3_0.1','closest_maplocation','tran_seq_500000_3_0.2',
                   'numero_del_referidos_de_persona_que_refirio', 'classification_risk']

    col_categoricas = ['fraude_general','ret_first_category','user_is_pep', 
                       'visa_currency_first_category','ret_second_category','visa_first_category',
                       'dep_second_category','url_type',
                       'device_type_description',
                       'tipo_registro_documento','dep_first_category',
                       'visa_spending_category',
                       'more_than_2_emails','more_than_2_phones','more_than_2_accounts']

    ##############################################################
    ## separate the dataframe in categorical and numeric variables.
    cat_df=data_fil[col_categoricas]
    num_df=data_fil[col_numericas]
    
    ## function to get variables with a high correlation
    high_correlations_table=pc.preproccessing_func.correlations_treatment(num_df,correlation_treshold)
    
    ##########################################

    ## depending on the last parameter we decide if we want to delete variables with a high correlation
    num_corr=high_correlations_table[list(high_correlations_table.index)]
    if del_high_corr:
        ## initialize the variables
        iterator=0
        columns_list=[]
        temp=num_corr.iloc[iterator:len(num_corr),iterator:len(num_corr)]

        ## extract variables that already contains information from other variables
        while iterator<(len(temp)-1):
            ## reducing the matrix according to the sections we already evaluated
            temp=temp.iloc[iterator:len(temp),iterator:len(temp)]

            ## find the columns that the actual column are most correlated with and update the iterator
            iterator=pc.preproccessing_func.corr_cought(temp.iloc[:,0].isna())

            ## save columns to be used
            columns_list.append(temp.columns.tolist()[0])

        ## final selection of numerical variables to be used.
        final_num_vars=list(set(num_df.columns)-set(list(high_correlations_table.index)))+columns_list
    else:
        final_num_vars=set(num_df.columns)
        
    ## leave the dataframe just with the variables we filtered
    num_df=num_df[final_num_vars]
    
    ## boolean as parameter to evaluate if we replace the concentrated columns with categoricals
    concentrated_treatment=True


    data_temp=num_df.copy(deep=True)
    if concentrated_treatment:
        # list of columns with the value they are most concentrated on
        variables_concentrated=pc.preproccessing_func.concentrated_transform(num_df,concent_treshold,['numero_de_referidos',
                                           'score_coink',
                                           'numero_del_referidos_de_persona_que_refirio',
                                           'num_otras_entidades_ACH',
                                           'num_remesas',
                                           'num_pse','num_reval_ptm','num_p2p',
                                           'cluster_number','user_phonenumber','user_email'])

        ## put 0 on the concentrated value and 1 in other values
        for i in variables_concentrated:
            data_temp[i[0]]=[0 if i[1]==j else 1 for j in data_temp[i[0]]]
    else:
        print('no_concentrated_treatment')
        
    #pass this numerical values to the categoricals
    cat_df=pd.concat([cat_df,data_temp[np.array(variables_concentrated)[:,0]]],axis=1)
    num_df=num_df[set(num_df.columns)-set(np.array(variables_concentrated)[:,0])]

    ## we want to treat outliers or not
    if treat_outliers:
        print(methodology)
        print(correction_type)
        print(perc_step)
        print(variables_exclude)
        print(variables)
        ## treat autliers with the percentiles methodology
        treated_data=pc.preproccessing_func.outliers_treatment(num_df,
                                       variables='todas',
                                       variables_exclude=[],
                                       perc_step=perc_step,
                                       correction_type="replace",
                                       exclude_negatives=False,
                                       methodology=methodology)
    else:
        treated_data=num_df.copy(deep=True)
        
    ## scale the numerical variables
    escalador = escalador
    escalador.fit(treated_data)
    treated_data_columnas=list(treated_data.columns)
    treated_data = escalador.transform(treated_data)

    ## pca treatment for categorical variables.
    if PCA_treatment:
        ## define the number of components
        pca = PCA(n_components=number_components)
        pca.fit(treated_data)

        data_pca=np.matmul(np.array(treated_data),pca.components_.T)
        treated_data=pd.DataFrame(data_pca,columns=["component"+str(i) for i in range(number_components)])
    else:
        treated_data=pd.DataFrame(treated_data,columns=treated_data_columnas)
        
    ###########################
    ## VARIABLES CATEGORICAS ##
    ###########################
    ## categorical columns to take into account in the model.
    selected_vars=set(cat_df.columns)-set(drop_vars)

    ## dataframe with just the vars that were selected
    cat_df=cat_df[selected_vars]
    
    #possibilities for the chi2 matrix
    comb_possibilities=list(combinations(list(cat_df.columns),2))
    
    
    ## create the matrix of correlation between categorical variables
    empty_list=pd.DataFrame(index=list(cat_df.columns),columns=list(cat_df.columns))
    for i in comb_possibilities:
        ## estimate the chi-squared test for categorical variables
        crosstab = pd.crosstab(cat_df[i[0]], cat_df[i[1]])
        chi2_met = stats.chi2_contingency(crosstab)[0]
        n = np.sum(np.array(crosstab))
        minDim = min(crosstab.shape)-1

        #calcule el V  de Cramer
        Cramer = np.sqrt((chi2_met/n)/minDim)

        empty_list.loc[[i[0]], [i[1]]]=Cramer

    ## fill the Na's in the DataFrame
    #empty_list.fillna(0,inplace=True)
    
    ## create a symetrical matrix with the Cramer index
    for i in empty_list.columns:
        empty_list[i]=empty_list[i].astype(float)

    ## Change the level of the index
    empty_list=((empty_list-1)*-1)
    empty_list_1=empty_list.fillna(0)

    ## make the matrix symetrical
    mat=np.array(empty_list_1)
    mat = mat + mat.T

    ## fill the diagonal of the matrix with 1
    np.fill_diagonal(mat,1)

    ## watch the correlation table and filter the correlated variables
    empty_list_1=pd.DataFrame(mat,columns=empty_list_1.columns,index=empty_list_1.columns)
    high_correlations_table_cat=pc.preproccessing_func.correlations_treatment(empty_list_1,cat_corr_tresh)
    
    ## depending on the last parameter we decide if we want to delete variables with a high correlation
    cat_corr=high_correlations_table_cat[list(high_correlations_table_cat.index)]
    
    ## initialize the variables to drop categorical variables with high correlations
    iterator=0
    columns_list=[]
    temp=cat_corr.iloc[iterator:len(cat_corr),iterator:len(cat_corr)]

    ## extract variables that already contains information from other variables
    while iterator<(len(temp)-1):
        ## reducing the matrix according to the sections we already evaluated
        temp=temp.iloc[iterator:len(temp),iterator:len(temp)]

        ## find the columns that the actual column are most correlated with and update the iterator
        iterator=pc.preproccessing_func.corr_cought(temp.iloc[:,0].isna())

        ## save columns to be used
        columns_list.append(temp.columns.tolist()[0])
        
    ## final selection of numerical variables to be used.
    final_cat_vars=list(set(cat_df.columns)-set(list(high_correlations_table_cat.index)))+columns_list
    
    ## leave just the variables we wanto to use for the model
    cat_df=cat_df[list(set(final_cat_vars)-set(["fraude_general"]))+["fraude_general"]]
    
    #############################
    ## one hot and complete df ##
    #############################
    
    ## get the dataframe processed
    categoricas = pd.get_dummies(cat_df, drop_first=True)
    df_usuarios_ohe = pd.concat([categoricas,treated_data], axis = 1)
    
    ## transform the boolean values to int
    df_usuarios_ohe['user_is_pep']=[1 if i else 0 for i in df_usuarios_ohe['user_is_pep']]
    
    return df_usuarios_ohe

In [4]:
import awswrangler as wr

In [5]:
data_pre = pd.read_csv(os.getcwd() + '/Data/Base_Consolidada/base_consolidada.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (80) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
# transactions = wr.s3.read_parquet('s3://cnk-datalake/transacciones/Output/notebook_elt_transacciones_overwrite/')

In [32]:
# transactions=transactions[transactions.operation_description=="Transacción aprobada"]
# transactions=transactions[~(transactions.type_description.isin(["ABONO DE INTERESES",
#                                                                 "TRASLADO DE SALDO POR CAMBIO DE PLAN",
#                                                                 "APLICAR PROMOCION COFRES","FIDELIZACION CLIENTES",
#                                                                 "PREMIO POR REFERIDOS","AJUSTE CREDITO VISA FISICA POS",
#                                                                 "NOTA CREDITO"]))]

In [36]:
# ## porcentaje del dinero que les entra por p2p
# dictionary_p2p=dict(transactions.groupby(["user_id"])[["operation_value","aum_filter"]].apply(lambda x:
#                                     (x[(x["operation_value"]>=0) & (x['aum_filter']=='TRANSFERENCIA NATURAL A NATURAL')]['operation_value'].sum())/x[(x["operation_value"]>=0)]['operation_value'].sum()))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning

In [38]:
# data_pre["p2p_cash_in"]=[dictionary_p2p[i] if z else 0 for i,z in zip(data_pre["user_id"],data_pre["user_id"].isin(list(dictionary_p2p.keys())))]

In [42]:
import awswrangler as wr
from datetime import datetime,timedelta

In [15]:
sens_active = wr.s3.read_csv(path='s3://coink-data-users-prod/active_users_data.csv')
sens_canceled = wr.s3.read_csv(path='s3://coink-data-users-prod/canceled_users_data.csv')
sens = pd.concat([sens_active, sens_canceled], axis=0)
sens['user_createddate'] = [(datetime.strptime(str(x)[0:18], '%Y-%m-%d %H:%M:%S') - timedelta(hours=5, minutes=0)).strftime("%Y-%m-%d %H:%M:%S") for x in sens['user_createddate']]

In [16]:
sens=sens[sens["user_id"].isin(data_pre[data_pre["fraude_general"]==1]["user_id"].unique())].sort_values("DOCUMENTO CLIENTE")

In [43]:
!pip install imblearn
from imblearn.over_sampling import SMOTE

## params to test the best preprocessing

In [44]:
pct_nan_list=[0.2]
correlation_teshold_list=[0.9]
del_high_corr_list=[True]
concentrated_treatment_list=[False]
concent_treshold_list=[0.99]
treat_outliers_list=[False]
variables_list=['todas']## we can pass todas so it takes all the data frame or the list of variables we want to use
variables_exclude_list=[[]]## we can exclude some variables from the dataset
perc_step_list=[1] ## how many percent will be reach bin when dividing the dataset. 
correction_type_list=['replace']## it can be replace or exlude depending on the method you want to use
exclude_negatives_list=[False] ## if you want to replace the negatives with missing values
methodology_list=['percentile'] ## methodology could be "percentile" or "std"
std_integer_list=[2]
escalador_list=[MinMaxScaler()]
PCA_treatment_list=[True,False]
number_components_list=[20]
drop_vars_list=[["visa_second_category","maplocation_name_frecuency",
        "ecosystem_name_frecuency","dep_second_category","departamento"]]
cat_corr_tresh_list=[0.8]

In [45]:
results_list_data=pd.read_csv('results_temp.csv',index_col=0)
results_list=np.array(results_list_data).tolist()

In [46]:
results_list[21]

["[0.2, 0.9, True, False, 0.99, False, 'todas', [], 1, 'replace', False, 'percentile', 2, MinMaxScaler(), False, 10, ['visa_second_category', 'maplocation_name_frecuency', 'ecosystem_name_frecuency', 'dep_second_category', 'departamento'], 0.8]",
 0.9845822281167108,
 0.9533923303834808]

In [50]:
## run the data with the best's results parameters.
data__post_pre=finale_preprocessing_data(data=data_pre,
                                         pct_nan=0.2,
                                         correlation_treshold=0.9,
                                         del_high_corr=True,
                                         concentrated_treatment=False,
                                         concent_treshold=0.99,
                                         treat_outliers=False,
                                         variables='todas',## we can pass todas so it takes all the data frame or the list of variables we want to use
                                         variables_exclude=[],## we can exclude some variables from the dataset
                                         perc_step=1, ## how many percent will be reach bin when dividing the dataset. 
                                         correction_type='replace',## it can be replace or exlude depending on the method you want to use
                                         exclude_negatives=False, ## if you want to replace the negatives with missing values
                                         methodology='percentile', ## methodology could be "percentile" or "std"
                                         std_integer=2,
                                         escalador=MinMaxScaler(),
                                         PCA_treatment=False,
                                         number_components=10,
                                         drop_vars=['visa_second_category', 'maplocation_name_frecuency', 'ecosystem_name_frecuency', 'dep_second_category', 'departamento'],
                                         cat_corr_tresh=0.8)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [51]:
data__post_pre

,corresp_cashout,more_than_2_accounts,num_p2p_users,pct_mean_withd,cajero_cashout,pct_balance_min,numero_rechazadas,p2p_cash_in,numero_sospechas,pocket_goals_count,pct_mean_withd_caj,tran_seq_750000_7_0.1,pse_cashin,more_than_2_emails,time_diff_tran_norm_den,more_than_2_phones,user_is_pep,fraude_general,visa_currency_first_category_ARS,visa_currency_first_category_AUD,visa_currency_first_category_BOB,visa_currency_first_category_BRL,visa_currency_first_category_CAD,visa_currency_first_category_CLP,visa_currency_first_category_COP,visa_currency_first_category_DOP,visa_currency_first_category_EUR,visa_currency_first_category_GBP,visa_currency_first_category_ILS,visa_currency_first_category_JPY,visa_currency_first_category_MAD,visa_currency_first_category_MXN,visa_currency_first_category_PEN,visa_currency_first_category_PYG,visa_currency_first_category_TRY,visa_currency_first_category_USD,visa_currency_first_category_VES,visa_currency_first_category_sin oper visa,ret_first_category_PAGO DE SERVICIOS,ret_first_category_PAGO PRESTAMO,ret_first_category_PAGO SOLICITUD TARJETA VISA DEBITO DIGITAL,ret_first_category_PAGO TARJETA VISA DEBITO DIGITAL,ret_first_category_PAGO TARJETA VISA DEBITO FISICA,ret_first_category_RETIRO CAJERO ELECTRONICO VISA FISICA,ret_first_category_RETIRO CORRESPONSAL NO PROPIO,ret_first_category_RETIRO CORRESPONSAL PROPIO,ret_first_category_RETIRO Y PAGO EN EFECTIVO CORRESPONSAL,ret_first_category_RETIRO Y PAGO EN EFECTIVO CORRESPONSAL NO PROPIO,ret_first_category_SOLICITUD DE DINERO (ENTREGA),ret_first_category_TRANSFERENCIA A CUENTA COINK,ret_first_category_TRANSFERENCIA A OTRAS ENTIDADES,ret_first_category_TRASLADO DE SALDO POR CAMBIO DE PLAN,ret_first_category_sin ret,visa_spending_category_AGRICULTURAL CO-OPERATIVES,visa_spending_category_AIRLINES AND AIR CARRIERS,visa_spending_category_AMBULANCE SERVICES,visa_spending_category_AMERICAN AIRLINES,"visa_spending_category_AMUSEMENT PARKS, CIRCUSES, CARNIVALS, AND FORTUNE TELLERS","visa_spending_category_ANTIQUE SHOPS - SALES, REPAIRS, AND RESTORATION SERVICES","visa_spending_category_APPLIANCE STORES, NOT ELSEWHERE CLASSIFIED, HOUSEHOLD APPLIANCE STORES",visa_spending_category_ART DEALERS AND GALLERIES,visa_spending_category_AUTOMATED FUEL DISPENSERS,visa_spending_category_AUTOMOBILE RENTAL AGENCY,visa_spending_category_AUTOMOTIVE PARTS AND ACCESSORIES STORES,visa_spending_category_AUTOMOTIVE SERVICE SHOPS (NON-DEALER),visa_spending_category_AUTOMOTIVE TIRE STORES,visa_spending_category_AVIANCA,visa_spending_category_BAKERIES,"visa_spending_category_BANDS, ORCHESTRAS, AND MISCELLANEOUS ENTERTAINERS, NOT ELSEWHERE CLASSIFIED",visa_spending_category_BEAUTY AND BARBER SHOPS,"visa_spending_category_BETTING, INCLUDING LOTTERY TICKETS, CASINO GAMING CHIPS, OFF-TRACK BETTING, AND WAGERS AT RACE TRACKS",visa_spending_category_BICYCLE SHOPS - SALES AND SERVICE,visa_spending_category_BOOK STORES,"visa_spending_category_BOOKS, PERIODICALS, AND NEWSPAPERS",visa_spending_category_BUS LINES,"visa_spending_category_BUSINESS SERVICES, NOT ELSEWHERE CLASSIFIED","visa_spending_category_CABLE, SATELLITE, AND OTHER PAY TELEVISION AND RADIO SERVICES",visa_spending_category_CAMERA AND PHOTOGRAPHY SUPPLY STORES,"visa_spending_category_CANDY, NUT, AND CONFECTIONERY STORES","visa_spending_category_CAR AND TRUCK DEALERS (NEW AND USED) - SALES, SERVICE, REPAIRS, AND LEASING",visa_spending_category_CATERERS,visa_spending_category_CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS,visa_spending_category_CHILDREN S AND INFANT S WEAR,visa_spending_category_CIGAR STORES AND STANDS,visa_spending_category_CLEANING AND MAINTENANCE JANITORIAL SERVICES,"visa_spending_category_CLOTHING - FAMILY, FAMILY CLOTHING STORES","visa_spending_category_COLLEGES, UNIVERSITIES, PROFESSIONAL SCHOOLS, AND JUNIOR COLLEGES",visa_spending_category_COMMERCIAL FOOTWEAR,"visa_spending_category_COMMERCIAL SPORTS, PROFESSIONAL SPORTS CLUBS, ATHLETIC FIELDS, AND SPORTS PROMOTERS","visa_spending_category_COMPUTER MAINTENANCE

In [52]:
## save the date in s3 so we can run the models
data__post_pre.to_parquet(os.getcwd() + '/Data/Base_One_Hot_Encoding/base_one_hot_encoding.parquet')

In [39]:
iteration=0
# results_list=[]
for pct_nan in pct_nan_list:
    for correlation_teshold in correlation_teshold_list:
        for del_high_corr in del_high_corr_list:
            for concentrated_treatment in concentrated_treatment_list:
                for concent_treshold in concent_treshold_list:
                    for treat_outliers in treat_outliers_list:
                        for variables in variables_list:
                            for variables_exclude in variables_exclude_list:
                                for perc_step in perc_step_list:
                                    for correction_type in correction_type_list:
                                        for exclude_negatives in exclude_negatives_list:
                                            for methodology in methodology_list:
                                                for std_integer in std_integer_list:
                                                    for escalador in escalador_list:
                                                        for PCA_treatment in PCA_treatment_list:
                                                            for number_components in number_components_list:
                                                                for drop_vars in drop_vars_list:
                                                                    for cat_corr_tresh in cat_corr_tresh_list:
                                                                        ## in case the loop fails
                                                                        iteration+=1
                                                                        
                                                                        print(i)
                                                                        
                                                                        if iteration>0:
                                                                            ## preprocessing the data with the grid of parameters
                                                                            data__post_pre=finale_preprocessing_data(data=data_pre,
                                                                                                                     pct_nan=pct_nan,
                                                                                                                     correlation_treshold=correlation_teshold,
                                                                                                                     del_high_corr=del_high_corr,
                                                                                                                     concentrated_treatment=concentrated_treatment,
                                                                                                                     concent_treshold=concent_treshold,
                                                                                                                     treat_outliers=treat_outliers,
                                                                                                                     variables='todas',## we can pass todas so it takes all the data frame or the list of variables we want to use
                                                                                                                     variables_exclude=[],## we can exclude some variables from the dataset
                                                                                                                     perc_step=perc_step, ## how many percent will be reach bin when dividing the dataset. 
                                                                                                                     correction_type='replace',## it can be replace or exlude depending on the method you want to use
                                                                                                                     exclude_negatives=False, ## if you want to replace the negatives with missing values
                                                                                                                     methodology=methodology, ## methodology could be "percentile" or "std"
                                                                                                                     std_integer=std_integer,
                                                                                                                     escalador=escalador,
                                                                                                                     PCA_treatment=PCA_treatment,
                                                                                                                     number_components=number_components,
                                                                                                                     drop_vars=drop_vars,
                                                                                                                     cat_corr_tresh=cat_corr_tresh)

                                                                            ## dividing training and testing dataset
                                                                            X = data__post_pre.drop('fraude_general', axis = 1)
                                                                            y = data__post_pre['fraude_general']
                                                                            x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

                                                                            ## param to define if we want to do smote or not
                                                                            SMOTE_treatment=True

                                                                            #function to make smote
                                                                            if SMOTE_treatment:
                                                                                sm = SMOTE(random_state=42)
                                                                                X_train, y_train = sm.fit_resample(x_train.values, y_train.values)
                                                                            else:
                                                                                X_train=x_train.copy(deep=True).values

                                                                            ## estimating the model and saving the results
                                                                            gb_model = GradientBoostingClassifier()
                                                                            gb_model.fit(X_train,y_train)
                                                                            preds = gb_model.predict(x_test)

                                                                            mat_2=confusion_matrix(y_test,preds)

                                                                            ## results for each class
                                                                            recall_norm=mat_2[0,0]/(mat_2[0,1]+mat_2[0,0])
                                                                            recall_fraud=mat_2[1,1]/(mat_2[1,1]+mat_2[1,0])

                                                                            results_list.append([[pct_nan,correlation_teshold,del_high_corr,
                                                                                                  concentrated_treatment,concent_treshold,treat_outliers,
                                                                                                  variables,variables_exclude,perc_step,correction_type,
                                                                                                  exclude_negatives,methodology,std_integer,
                                                                                                  escalador,PCA_treatment,number_components,
                                                                                                  drop_vars,cat_corr_tresh],
                                                                                                  recall_norm,recall_fraud])
                                                                        else:
                                                                            print_("stored")

30


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


30


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [31]:
list_resultados=[]
for i in range(len(results_list)):
    list_resultados.append([i,results_list[i][2]])

In [36]:
results_list[21]

["[0.2, 0.9, True, False, 0.99, False, 'todas', [], 1, 'replace', False, 'percentile', 2, MinMaxScaler(), False, 10, ['visa_second_category', 'maplocation_name_frecuency', 'ecosystem_name_frecuency', 'dep_second_category', 'departamento'], 0.8]",
 0.9845822281167108,
 0.9533923303834808]

In [35]:
list_resultados

[[0, 0.9504424778761059],
 [1, 0.9504424778761059],
 [2, 0.9522123893805308],
 [3, 0.9522123893805308],
 [4, 0.9504424778761059],
 [5, 0.9504424778761059],
 [6, 0.9522123893805308],
 [7, 0.9522123893805308],
 [8, 0.9498525073746312],
 [9, 0.9533923303834808],
 [10, 0.9498525073746312],
 [11, 0.9533923303834808],
 [12, 0.6135693215339233],
 [13, 0.671976401179941],
 [14, 0.951622418879056],
 [15, 0.952802359882006],
 [16, 0.9498525073746312],
 [17, 0.9533923303834808],
 [18, 0.9498525073746312],
 [19, 0.9533923303834808],
 [20, 0.9498525073746312],
 [21, 0.9533923303834808],
 [22, 0.9498525073746312],
 [23, 0.9533923303834808],
 [24, 0.6224188790560472],
 [25, 0.6943952802359882],
 [26, 0.951622418879056],
 [27, 0.952802359882006],
 [28, 0.6135693215339233],
 [29, 0.671976401179941],
 [30, 0.951622418879056]]

In [55]:
pd.DataFrame(results_list).to_csv('results_temp.csv')